# inference

langchain을 통한 inference

https://python.langchain.com/docs/integrations/llms/llamacpp

## Langchain - SOLAR, Mistral

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

In [ ]:
n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Make sure the model path is correct for your system!
solar = LlamaCpp(
    model_path="./solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    temperature=0.6,
    top_p=1,
    max_tokens=128,
    stop=["Person1:", "Person2:"],
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [ ]:
# Make sure the model path is correct for your system!
mistral = LlamaCpp(
    model_path="./mistral/ggml-model-q5_k_m.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    temperature=0.6,
    top_p=1,
    max_tokens=128,
    stop=["Person1:", "Person2:"],
    verbose=True,  # Verbose is required to pass to the callback manager
)

## GPTQ - Polyglot

https://velog.io/@acdongpgm/LLM.-%ED%95%9C%EA%B5%AD%EC%96%B4-LLM-%EB%AA%A8%EB%8D%B8-GPTQ%EB%A1%9C-%EC%96%91%EC%9E%90%ED%99%94-%ED%95%98%EA%B8%B0-feat.-koalpaca-kullm

https://huggingface.co/j5ng/kullm-12.8b-GPTQ-8bit

GPU 메모리를 70GB를 차지했던 kullm 모델을 8bit로 양자화하여 13GB만 가지고도 사용할 수 있었습니다. 성능 저하를 우려했으나, 테스트 결과가 동일했습니다.

*4bit 양자화도 진행했으나, 외계어를 생성했습니다. 4bit는 욕심인가 봅니다.

실제 커뮤니티에 질문했을때 , 8bit 양자화했을때 결과가 똑같은 것 만으로도 기적이라고 합니다.

In [ ]:
"""
A dedicated helper to manage templates and prompt building.
"""

import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

In [ ]:
import torch
from transformers import pipeline
from auto_gptq import AutoGPTQForCausalLM


MODEL = "j5ng/kullm-12.8b-GPTQ-8bit"
model = AutoGPTQForCausalLM.from_quantized(MODEL, device="cuda:0", use_triton=False, cache_dir='kullm')

pipe = pipeline('text-generation', model=model,tokenizer=MODEL)

prompter = Prompter("kullm")

GPTQ ERROR: CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. 

> https://github.com/AutoGPTQ/AutoGPTQ




In [ ]:
def infer(instruction="", input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(
        prompt, max_length=512,
        temperature=0.2,
        repetition_penalty=3.0,
        num_beams=5,
        eos_token_id=2
    )
    s = output[0]["generated_text"]
    result = prompter.get_response(s)

    return result

In [ ]:
instruction = """
손흥민(한국 한자: 孫興慜, 1992년 7월 8일 ~ )은 대한민국의 축구 선수로 현재 잉글랜드 프리미어리그 토트넘 홋스퍼에서 윙어로 활약하고 있다.
또한 대한민국 축구 국가대표팀의 주장이자 2018년 아시안 게임 금메달리스트이며 영국에서는 애칭인 "쏘니"(Sonny)로 불린다.
아시아 선수로서는 역대 최초로 프리미어리그 공식 베스트 일레븐과 아시아 선수 최초의 프리미어리그 득점왕은 물론 FIFA 푸스카스상까지 휩쓸었고 2022년에는 축구 선수로는 최초로 체육훈장 청룡장 수훈자가 되었다.
손흥민은 현재 리그 100호를 넣어서 화제가 되고 있다.
"""
result = infer(instruction=instruction, input_text="손흥민의 애칭은 뭐야?")
print(result)

result = infer(instruction=instruction, input_text="손흥민의 포지션은 어디야?")
print(result)

## KULLM

- 12.8b - 8bit

4-bit 이하에서는 성능이 현저히 떨어진다고 함 -> 8-bit 기준 GPU 19gb 점유하므로 NCP 서버 단에서 돌릴 것

### utils

In [1]:
"""
A dedicated helper to manage templates and prompt building.
"""

import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

### inference

5.8b

In [2]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    cache_dir='./cache'
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline("text-generation", model=model, tokenizer=MODEL, device=0)

prompter = Prompter("kullm")


def infer(instruction="", input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
    s = output[0]["generated_text"]
    result = prompter.get_response(s)

    return result


result = infer(input_text="고려대학교에 대해서 알려줘")
print(result)
# '고려대학교에 대해 궁금한 점이 있으시면 언제든지 문의해 주세요. 고려대학교는 한국에서 가장 오래되고 권위 있는 대학교 중 하나로, 고려대학교의 역사는 한국의 역사와 함께해 왔습니다. 고려대학교는 학문적 우수성을 추구하는 동시에 사회적 책임을 다하기 위해 최선을 다하고 있습니다. 고려대학교는 학생, 교수진, 교직원을 위한 다양한 프로그램과 지원을 제공하는 것으로 유명합니다. 고려대학교는 한국의 정치, 경제, 사회 분야에서 중요한 역할을 담당하고 있습니다. 고려대학교에 대해 더 자세히 알고 싶으신가요?'

c:\Users\hwyew\miniconda3\envs\true_friend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:24<00:00,  8.00s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\hwyew\miniconda3\envs\true_friend\lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `tem

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 11.16 GiB is allocated by PyTorch, and 25.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

AssertionError: Torch not compiled with CUDA enabled

torch version 다시 맞춰주자
(https://pytorch.org/get-started/locally/)

## Prompt Experiment

### Experiment - 1

In [ ]:
template = "주어진 Persona를 가진 사람으로 1개의 문장으로 된 답변을 생성해주세요.\nPersona:{context_list}\ndialog:{src_list}\nPerson2:"
prompt = PromptTemplate.from_template(template)
prompt

In [ ]:
src_list = '\nPerson1:안녕하십니까! 저는 20대 여자입니다~\nPerson2:네~즐거운 대화해요~!! 저는 40대 여자입니다~\nPerson1:무슨일 하시나요? 저는 중학교 수학교인데, 요즘 바쁜 시기에요.'
context_list = '나는 여자이다.\n나는 40대이다.\n나는 가족과 함께하는 시간이 많다.\n나는 비글 두 마리를 키운다.\n나는 요즘, 자주 상처를 받는다.\n나는 40대 여자이다.'

In [ ]:
solar_chain = LLMChain(prompt=prompt, llm=solar)
prediction = solar_chain.run({'src_list' : src_list, 'context_list' : context_list})

In [ ]:
mistral_chain = LLMChain(prompt=prompt, llm=mistral)
prediction = mistral_chain.run({'src_list' : src_list, 'context_list' : context_list})

### Experiment - 2

In [ ]:
template = "주어진 페르소나를 가진 사람이 되어 이전 대화 맥락에 맞게 답변해주세요. \n페르소나:{context_list}\n이전 대화:{src_list}\nPerson2:"
prompt = PromptTemplate.from_template(template)

In [ ]:
src_list = '\nPerson1:안녕하십니까! 저는 20대 여자입니다~\nPerson2:네~즐거운 대화해요~!! 저는 40대 여자입니다~\nPerson1:무슨일 하시나요? 저는 중학교 수학교인데, 요즘 바쁜 시기에요.'
context_list = '나는 여자이다.\n나는 40대이다.\n나는 가족과 함께하는 시간이 많다.\n나는 비글 두 마리를 키운다.\n나는 요즘, 자주 상처를 받는다.\n나는 40대 여자이다.'

In [ ]:
solar_chain = LLMChain(prompt=prompt, llm=solar)
solar_chain.run({'src_list' : src_list, 'context_list' : context_list})